# R3: Recall, Rank, Rerank

In [5]:
import os
import json
from pprint import pprint

os.chdir("/root/xiatian/github/llms4subjects")

In [ ]:
import json
from llms4subjects.instance import instance_db_merged_with_dev
from llms4subjects.subject import subject_db_all

predicted_file = "./db/eval/merged/by_instance_5.dev2.jsonline"
with open(predicted_file, "r", encoding="utf-8") as f:
    records = [json.loads(line) for line in f.readlines()]
    
for r in records:
    instance = instance_db_merged_with_dev.get_by_instance_id(r['id'])
    r['true_names'] = [subject_db_all.get_name_by_code(name) for name in r['true_codes']]
    r['pred_names'] = [subject_db_all.get_name_by_code(name) for name in r['pred_codes']]
    r['abstract'] = instance.abstract
    r['doctype'] = instance.doctype
    r['language'] = instance.language
    
# 写回jsonline文件
with open('./db/eval/merged/by_instance_5.dev2_with_names.jsonline', "w", encoding="utf-8") as f:
    for r in records:
        line = json.dumps(r,ensure_ascii=False)
        f.write(f"{line}\n")
    

load 112292 alias
load 311410 alias


In [6]:
# 重新读取
with open('./db/eval/merged/by_instance_5.dev2_with_names.jsonline', "r", encoding="utf-8") as f:
    records = [json.loads(line) for line in f.readlines()]
    

In [ ]:
template_v1 = """
You act as an expert in library subject indexing. Please carefully analyze the given document title and abstract, review the given list of reference topics, and reorder them according to their degree of relevance to the document. You can eliminate irrelevant topics and also add new topics.

## Title: xxxx
## Abstract: xxxx
## Reference list of possible document topics for sorting:
	- Topic 1
	- Topic 2

## Please re-output the sorted list of document topics: 
"""


tempate_v2 = """You act as an expert in library subject indexing. Please carefully analyze the given document title and abstract, review the given list of reference topics, and reorder them according to their degree of relevance to the document. Irrelevant topics can be removed, and new topics can also be added.

## Here is an example of the input and output format
### Title: xxxx
### Abstract: xxxx
### Reference sorted list of document topics:
  - Topic 1
  - Topic 2
  
### Analysis process
(omitted)

### Final topic list
  - Topic 1
  - Topic 2

## Normal processing starts here

### Title: {your title}
### Abstract: {your abstract}
### Reference sorted list of document topics:
  - {your topic 1}
  - {your topic 2}
  
### Analysis process"""
def make_prompt(record):
    topics = [f"  - {name.strip()}" for name in record["pred_names"]]
    topics = "\n".join(topics)
    title, abstract = record["title"], record["abstract"]
    
    return f"""
You act as an expert in library subject indexing. Please carefully analyze the given document title and abstract, review the given list of reference topics, and reorder them according to their degree of relevance to the document. Irrelevant topics can be removed, and new topics can also be added.

## Here is an example of the input and output format
### Title: xxxx
### Abstract: xxxx
### Reference sorted list of document topics:
  - Topic 1
  - Topic 2
    
### Analysis process
(omitted)

### Final topic list
  - Topic 1
  - Topic 2

## Normal processing starts here

### Title: {title}
### Abstract: {abstract}
### Reference sorted list of document topics:
{topics}

### Analysis process
"""

In [24]:
p1 = make_prompt(records[6])
print(p1)
pprint(records[6])


You act as an expert in library subject indexing. Please carefully analyze the given document title and abstract, review the given list of reference topics, and reorder them according to their degree of relevance to the document. Irrelevant topics can be removed, and new topics can also be added.

## Here is an example of the input and output format
### Title: xxxx
### Abstract: xxxx
### Reference sorted list of document topics:
  - Topic 1
  - Topic 2
    
### Analysis process
(omitted)

### Final topic list
  - Topic 1
  - Topic 2

## Normal processing starts here

### Title: Aktuelle Entwicklungen im Immissionsschutzrecht : Dokumentation des 17. Leipziger Umweltrechtlichen Symposions des Instituts für Umwelt- und Planungsrecht der Universität Leipzig und des Helmholtz-Zentrums für Umweltforschung - UFZ am 19. und 20. April 2012
### Abstract: Anlässlich der Umsetzung der europäischen Industrieemissionsrichtlinie in nationales Recht stellen sich eine Reihe von Fragen, insbesondere zu

Familienbetrieb
Beirat
Corporate Governance
Familiengesellschaft
Gesellschafter
Nachhaltigkeit
Verantwortung